In [1]:
import pandas as pd
from unidecode import unidecode
from helper.df_handler import kor_get_last, eng_get_first_and_last, kor_update_last, kor_update_first, kor_handle_u, manual_check, eng_remove_jr, manual_merge, eng_handle_rare_last_name, recover_dash, eng_check_possible_match
from helper.ref_dict import dob_dict_non_kr

In [2]:
df_st = pd.read_csv('../data/st_player.csv')
df_br = pd.read_csv('../data/br_player.csv')

In [3]:
# data normalization (statiz)
df_st['name_eng'] = df_st['name_eng'].str.strip()             # remove whitespace around the name
df_st['name_eng'] = df_st['name_eng'].str.replace('--', '-')  # change '--' to '-'
df_st['name_eng'] = df_st['name_eng'].apply(recover_dash)     # manual addition of '-' for those who are missing it
df_st['name_new'] = df_st['name_eng'].str.lower()             # lower all characters.
df_st['name_new'] = df_st['name_new'].str.replace('-', ' ')   # remove '-'.
df_st['name_new'] = df_st['name_new'].apply(unidecode)        # decode with unicode to remove accents.
df_st = df_st.apply(manual_check, axis=1)
# print(df_st.head(5))
# df_st['name_new'] = df_st['name_new'].apply(manual_check)     # manual manipulation for unconventional cases (order matters! must be here)
df_st['name_new'] = df_st['name_new'].str.replace('young', 'yeong')
df_st['name_new'] = df_st['name_new'].str.replace('yung', 'yeong')
df_st['name_new'] = df_st['name_new'].str.replace('  ', ' ')  # change '  ' to ' '
df_st['name_new'] = df_st['name_new'].apply(kor_update_last)  # generalize the last name
df_st['name_new'] = df_st['name_new'].apply(kor_update_first) # generalize the first name
df_st['name_new'] = df_st['name_new'].apply(eng_remove_jr)    # remove 'Jr' at the end
df_st['date_of_birth'] = df_st.apply(lambda row: dob_dict_non_kr[row['name_eng']] if row['name_eng'] in dob_dict_non_kr else row['date_of_birth'], axis=1) # birthday manipulation

# print(df_st.head(5))

In [4]:
correct_data = ("Jesus Sanchez", "1974-10-11", "https://www.baseball-reference.com/players/s/sanchje01.shtml")
df_br.loc[df_br['name_eng'] == "Micah Bowie", :] = correct_data

In [5]:
# data normalization (baseball reference)
df_br['name_eng'] = df_br['name_eng'].str.strip()             # remove whitespace around the name
df_br['name_eng'] = df_br['name_eng'].str.replace('--', '-')  # change '--' to '-'
df_br['name_eng'] = df_br['name_eng'].apply(recover_dash)     # manual addition of '-' for those who are missing it
df_br['name_new'] = df_br['name_eng'].str.lower()             # lower all characters.
df_br['name_new'] = df_br['name_new'].str.replace('-', ' ')   # remove '-'.
df_br['name_new'] = df_br['name_new'].apply(unidecode)        # decode with unicode to remove accents.
df_br = df_br.apply(manual_check, axis=1)
# df_br['name_new'] = df_br['name_new'].apply(manual_check)     # manual manipulation for unconventional cases (order matters! must be here)
df_br['name_new'] = df_br['name_new'].str.replace('young', 'yeong')
df_br['name_new'] = df_br['name_new'].str.replace('yung', 'yeong')
df_br['name_new'] = df_br['name_new'].str.replace('  ', ' ')  # change '  ' to ' '
df_br['name_new'] = df_br['name_new'].apply(kor_update_last)  # generalize the Korean last name
df_br['name_new'] = df_br['name_new'].apply(kor_update_first) # generalize the Korean first name
df_br['name_new'] = df_br['name_new'].apply(eng_remove_jr)    # remove 'Jr' at the end

print(df_br.head(5))

           name_eng date_of_birth  \
0     Seung Han Ahn    1992-01-25   
1    Raúl Alcántara    1992-12-04   
2  Jordan Balazovic    1998-09-17   
3     Kyu Bin Chang    2001-04-21   
4      Ji Kang Choi    2001-07-23   

                                                 url          name_new  
0  https://www.baseball-reference.com/register/pl...     seung han ahn  
1  https://www.baseball-reference.com/register/pl...    raul alcantara  
2  https://www.baseball-reference.com/register/pl...  jordan balazovic  
3  https://www.baseball-reference.com/register/pl...      kyu bin jang  
4  https://www.baseball-reference.com/register/pl...      ji kang choi  


In [6]:
inner_join = pd.merge(df_br, df_st, on=['name_new', 'date_of_birth'], how='inner')
print(inner_join.head(5))
print(len(inner_join))

         name_eng_x date_of_birth  \
0     Seung Han Ahn    1992-01-25   
1    Raúl Alcántara    1992-12-04   
2  Jordan Balazovic    1998-09-17   
3     Kyu Bin Chang    2001-04-21   
4      Ji Kang Choi    2001-07-23   

                                               url_x          name_new  \
0  https://www.baseball-reference.com/register/pl...     seung han ahn   
1  https://www.baseball-reference.com/register/pl...    raul alcantara   
2  https://www.baseball-reference.com/register/pl...  jordan balazovic   
3  https://www.baseball-reference.com/register/pl...      kyu bin jang   
4  https://www.baseball-reference.com/register/pl...      ji kang choi   

  name_kor        name_eng_y  \
0      안승한     Seung-Han Ahn   
1     알칸타라    Raul Alcántara   
2     발라조빅  Jordan Balazovic   
3      장규빈      Gyu-Bin Jang   
4      최지강      Ji-Kang Choi   

                                               url_y  
0  https://statiz.sporki.com/player/?m=playerinfo...  
1  https://statiz.sporki.com/

In [7]:
# outer_join is only to pick up left over. 
outer_join = pd.merge(df_br, df_st, on=['name_new', 'date_of_birth'], how='outer')
# outer_join = outer_join.drop(columns=['url'])
outer_join = outer_join[outer_join.isna().any(axis=1)]
print(outer_join.head(5))
print(len(outer_join))

     name_eng_x date_of_birth  \
2  Aaron Brooks    1990-04-27   
3           NaN    1990-04-27   
5           NaN    1991-10-03   
6   Adam Plutko    1991-10-03   
7           NaN    1987-12-09   

                                               url_x             name_new  \
2  https://www.baseball-reference.com/register/pl...         aaron brooks   
3                                                NaN     aaron lee brooks   
5                                                NaN  adam gregory plutko   
6  https://www.baseball-reference.com/register/pl...          adam plutko   
7                                                NaN     adam robert wilk   

  name_kor           name_eng_y  \
2      NaN                  NaN   
3      브룩스     Aaron Lee Brooks   
5      플럿코  Adam Gregory Plutko   
6      NaN                  NaN   
7       아담     Adam Robert Wilk   

                                               url_y  
2                                                NaN  
3  https://statiz

In [8]:
inner_join, outer_join = manual_merge(inner_join, outer_join, kor_handle_u)

given name: amaury telemaco -> neither exists
given name: arquimedez pozo -> neither exists
given name: austin dean -> neither exists
given name: beau sulser -> neither exists
given name: beom jun park -> found the unique match
given name: beom su jeong -> found the unique match
given name: beom su kim -> found the unique match
given name: bubba carpenter -> neither exists
given name: bum ho lee -> found the unique match
given name: bum hyeon cho -> found the unique match
given name: burch smith -> neither exists
given name: chuck smith -> neither exists
given name: chun woong lee -> found the unique match
given name: dae su byeon -> found the unique match
given name: dae sun jeong -> found the unique match
given name: deok yun kim -> found the unique match
given name: deuk yum ka -> found the unique match
given name: do yun kim -> found the unique match
given name: dong su choi -> found the unique match
given name: doug clark -> neither exists
given name: doug mathis -> neither exists

In [9]:
inner_join, outer_join = manual_merge(inner_join, outer_join, eng_handle_rare_last_name)

In [10]:
inner_join, outer_join = manual_merge(inner_join, outer_join, eng_check_possible_match)

given name: albert joe suarez -> found the unique match
given name: albert martin -> found the unique match
given name: andrew james anderson -> neither exists
given name: andrew james anderson -> found the unique match
given name: andrew jose suarez -> found the unique match
given name: angel luis sanchez -> found the unique match
given name: angel maria pena -> found the unique match
given name: ariel bolivar jurado agrazal -> found the unique match
given name: arquimedez pozo ortiz -> found the unique match
given name: austin james dean -> found the unique match
given name: bradley richard thomas -> neither exists
given name: bradley richard thomas -> found the unique match
given name: brent raymond bowers -> found the unique match
given name: burch taylor smith -> found the unique match
given name: cedrick jerome bowers -> found the unique match
given name: charles edward smith -> neither exists
given name: charles edward smith -> found the unique match
given name: daniel danny rio

In [11]:
inner_join, outer_join = manual_merge(inner_join, outer_join, eng_handle_rare_last_name)

In [12]:
inner_join.to_csv('../data/done_proto.csv', index=False)  
outer_join.to_csv('../data/to_do_proto.csv', index=False)  